In [1]:
# 比较两者第一天的肾脏状态
# 比较C2与C3药物使用率及使用剂量


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_rows = 300
import math
import datetime as dt
import itertools
import random
import scipy.stats as stats
import seaborn as sns

# 亚型导入+时间规定

In [3]:
df_type = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/01.MIMICIV_SAKI_trajCluster/df_mixAK_fea4_C3.csv")
df_type_filt = df_type.loc[:,["stay_id","groupHPD"]]
df_type_filt = df_type_filt.drop_duplicates()
all_lst = df_type_filt.stay_id.unique().tolist()
print("纳入患者数量：",len(all_lst))

df_type_filt["groupHPD"].value_counts()

纳入患者数量： 4713


2    3055
1    1298
3     360
Name: groupHPD, dtype: int64

In [4]:
data_dir = "/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/00.data_mimic//disease_definition/AKI/"
infile = data_dir + "sk_event_time.csv"
df_time = pd.read_csv(infile)
df_time = df_time[df_time["stay_id"].isin(all_lst)]
df_time = df_time[["stay_id","saki_onset"]]
df_time["saki_onset"] = pd.to_datetime(df_time["saki_onset"])
df_time.loc[:, 'saki_onset_a1d'] = df_time.saki_onset + dt.timedelta(hours=24)
df_time = pd.merge(df_type_filt,df_time,how="inner",on="stay_id")
df_time.head(1)

,stay_id,groupHPD,saki_onset,saki_onset_a1d
0,30003598,1,2189-04-15 03:04:00,2189-04-16 03:04:00


In [5]:
df_dict = pd.read_csv("/public/hanl/jupyter_dir/database/mimicIV20/raw/raw/icu/d_items.csv")
df_dict = df_dict[["itemid","label"]]
df_dict.columns = ["itemid","drug"]

# 加压药

In [6]:
df_vaso = pd.read_csv("/public/hanl/jupyter_dir/database/mimicIV20/clean/medication/vaso/vaso_with_type.csv")
df_vaso["starttime"] = pd.to_datetime(df_vaso["starttime"])
df_vaso = pd.merge(df_vaso,df_time,how="right",on="stay_id")
#df_vaso = df_vaso[(df_vaso["starttime"]<=df_vaso["saki_onset_a1d"])&(df_vaso["starttime"]>=df_vaso["saki_onset"])]
df_vaso = df_vaso[['stay_id','starttime', 'vaso_type', 'vaso_amount',  'groupHPD' ]]
df_vaso.columns = ['stay_id','starttime', 'drug', 'amount',  'groupHPD']
df_vaso.head()

,stay_id,starttime,drug,amount,groupHPD
0,30003598,2189-04-15 03:15:00,phenylephrine,0.340136,1
1,30003598,2189-04-14 20:52:00,phenylephrine,25.034014,1
2,30003598,2189-04-15 03:20:00,phenylephrine,13.547932,1
3,30003598,2189-04-15 08:52:00,phenylephrine,8.210970,1
4,30003598,2189-04-15 20:10:00,phenylephrine,6.689000,1


In [7]:
# 药物使用率
df_sum = df_vaso.groupby(["stay_id","drug"]).agg({"amount":"sum"}).reset_index()
df_sum["is_vaso"] = 1
df_stat_01 = df_sum.pivot(index="stay_id",columns="drug",values="is_vaso").reset_index()
df_stat_01 = pd.merge(df_stat_01,df_type_filt,how="right",on="stay_id")
df_stat_01 = df_stat_01.fillna(0)
df_stat_01.groupby('groupHPD').mean()*100

,stay_id,dobutamine,dopamine,epinephrine,norepinephrine,phenylephrine,vasopressin
groupHPD,,,,,,,
1,3.488627e+09,2.542373,4.853621,10.939908,46.918336,39.599384,19.799692
2,3.508386e+09,0.916530,2.782324,8.379705,29.230769,38.101473,7.790507
3,3.521878e+09,5.555556,7.777778,16.944444,66.388889,51.666667,43.888889


In [8]:
# 药物平均使用量
df_stat_amount = df_sum.pivot(index="stay_id",columns="drug",values="amount").reset_index()
df_stat_amount = pd.merge(df_stat_amount,df_type_filt,how="inner",on="stay_id")
df_stat_amount.groupby('groupHPD').mean().reset_index()

,groupHPD,stay_id,dobutamine,dopamine,epinephrine,norepinephrine,phenylephrine,vasopressin
0,1,3.486334e+07,939.172661,736.183977,6.555991,39.334500,214.241707,131.968296
1,2,3.504296e+07,425.293955,698.746422,4.966266,23.459766,133.095336,98.135372
2,3,3.527019e+07,1306.727538,735.359192,15.529877,68.744696,371.049632,145.189239


# 抗生素

In [9]:
df_anti = pd.read_csv("/public/hanl/jupyter_dir/database/mimicIV20/clean/medication/antibiotic/antibiotic.csv")
df_anti["starttime"] = pd.to_datetime(df_anti["starttime"], format='%d/%m/%Y %H:%M:%S')
df_anti["stoptime"] = pd.to_datetime(df_anti["stoptime"], format='%d/%m/%Y %H:%M:%S')
df_anti = pd.merge(df_anti,df_time,how="right",on="stay_id")
# df_anti = df_anti[(df_anti["starttime"]<=df_anti["saki_onset_a1d"])&(df_anti["starttime"]>=df_anti["saki_onset"])]
df_anti = df_anti[["stay_id","starttime", "antibiotic","groupHPD"]]
df_anti.columns = ["stay_id","starttime", "drug","groupHPD"]
df_anti.head(1)

,stay_id,starttime,drug,groupHPD
0,30003598.0,2189-04-18 11:00:00,Linezolid,1


In [10]:
# 药物使用率
df_sum = df_anti.groupby(["stay_id","drug"]).agg({"groupHPD":"sum"}).reset_index()
df_sum["is_anti"] = 1
df_stat_01 = df_sum.pivot(index="stay_id",columns="drug",values="is_anti").reset_index()
df_stat_01 = pd.merge(df_stat_01,df_type_filt,how="right",on="stay_id")
df_stat_01 = df_stat_01.fillna(0)
df_stat_01 = df_stat_01.groupby('groupHPD').mean()*100
df_stat_01[df_stat_01 > 5].dropna(axis=1, how='all')


,stay_id,Ampicillin-Sulbactam,Azithromycin,Azithromycin,CeFAZolin,CefTAZidime,CefTRIAXone,CefazoLIN,CefePIME,CeftriaXONE,...,Ciprofloxacin IV,Levofloxacin,Meropenem,MetRONIDAZOLE (FLagyl),MetroNIDAZOLE,Mupirocin Ointment 2%,Piperacillin-Tazobactam,Tobramycin Sulfate,Vancomycin,Vancomycin Oral Liquid
groupHPD,,,,,,,,,,,,,,,,,,,,,
1,3.488627e+09,NaN,7.318952,7.473035,5.161787,5.546995,9.707242,18.952234,35.516179,12.557781,...,13.174114,12.018490,13.174114,17.180277,8.705701,NaN,30.046225,NaN,75.192604,7.318952
2,3.508386e+09,5.171849,6.284779,5.990180,7.790507,NaN,9.230769,27.103110,25.990180,12.307692,...,10.147300,10.212766,6.350245,13.158756,6.481178,6.08838,20.425532,NaN,65.106383,NaN
3,3.521878e+09,NaN,7.777778,7.777778,NaN,6.111111,7.777778,13.055556,42.777778,10.277778,...,16.666667,15.833333,16.666667,21.111111,13.611111,NaN,38.888889,6.111111,83.333333,10.833333


# 液体给入

In [11]:
df_fluid = pd.read_csv("/public/hanl/jupyter_dir/database/mimicIV20/clean/treatment/mimic_fluidin.csv")
df_fluid["starttime"] = pd.to_datetime(df_fluid["starttime"])
df_fluid = pd.merge(df_fluid,df_time,how="right",on="stay_id")
# df_fluid = df_fluid[(df_fluid["charttime"]<=df_fluid["saki_onset_a1d"])&(df_fluid["charttime"]>=df_fluid["saki_onset"])]

df_fluid =  df_fluid[["stay_id","starttime","label","amount","groupHPD"]]
df_fluid.columns = ["stay_id","starttime","drug","amount","groupHPD"]
df_fluid.head(1)

,stay_id,starttime,drug,amount,groupHPD
0,30003598,2189-04-14 23:31:00,Dextrose 5%,1.45,1


In [12]:
# # 药物使用率
# df_sum = df_fluid.groupby(["stay_id"]).agg({"amount":"sum"}).reset_index()
# #df_sum["drug"] ="fluid"
# df_sum["is_fluid"] = 1
# df_stat_01 = df_sum.pivot(index="stay_id",columns="drug",values="is_fluid").reset_index()
# df_stat_01 = pd.merge(df_stat_01,df_type_filt,how="right",on="stay_id")
# df_stat_01 = df_stat_01.fillna(0)
# df_stat_01.groupby('groupHPD').mean()*100

In [13]:
# # 药物平均使用量
# df_stat_amount = df_sum.pivot(index="stay_id",columns="drug",values="colloid_bolus_hourly").reset_index()
# df_stat_amount = pd.merge(df_stat_amount,df_type_filt,how="inner",on="stay_id")
# df_stat_amount.groupby('groupHPD').mean().reset_index()

# 利尿剂

In [14]:
df_di = pd.read_csv("/public/hanl/jupyter_dir/database/mimicIV20/clean/medication/diuretic/diuretic.csv")
df_di["starttime"] = pd.to_datetime(df_di["starttime"])
df_di = pd.merge(df_di,df_time,how="right",on="stay_id")
df_di = pd.merge(df_di,df_dict,how="inner",on="itemid")
# df_di = df_di[(df_di["starttime"]<=df_di["saki_onset_a1d"])&(df_di["starttime"]>=df_di["saki_onset"])]
df_di = df_di[[ 'stay_id', 'starttime',"drug", 'amount',"groupHPD" ]]
map_dict = {"Furosemide (Lasix)":"Furosemide", "Furosemide (Lasix) 250/50":"Furosemide","Bumetanide (Bumex)":"Bumetanide", "Mannitol":"Mannitol"}
df_di["drug"] = df_di["drug"].map(map_dict)
df_di.head(1)

,stay_id,starttime,drug,amount,groupHPD
0,30003598,2189-04-18 12:07:00,Furosemide,20.000001,1


In [15]:
df_sum = df_di.groupby(["stay_id","drug"]).agg({"amount":"sum"}).reset_index()
df_sum.head(1)

,stay_id,drug,amount
0,30003598,Furosemide,40.000003


In [16]:
# 药物使用率
df_sum["is_diuretic"] = 1
df_stat_01 = df_sum.pivot(index="stay_id",columns="drug",values="is_diuretic").reset_index()
df_stat_01 = pd.merge(df_stat_01,df_type_filt,how="right",on="stay_id")
df_stat_01 = df_stat_01.fillna(0)
df_stat_01.groupby('groupHPD').mean()*100

,stay_id,Bumetanide,Furosemide,Mannitol
groupHPD,,,,
1,3.488627e+09,0.616333,71.648690,0.924499
2,3.508386e+09,0.065466,63.175123,1.178396
3,3.521878e+09,0.555556,59.166667,1.944444


In [17]:
# 药物平均使用量
df_stat_amount = df_sum.pivot(index="stay_id",columns="drug",values="amount").reset_index()
df_stat_amount = pd.merge(df_stat_amount,df_type_filt,how="inner",on="stay_id")
df_stat_amount.groupby('groupHPD').mean().reset_index()

,groupHPD,stay_id,Bumetanide,Furosemide,Mannitol
0,1,3.494037e+07,97.995136,431.180279,183.124997
1,2,3.510395e+07,16.385417,191.797706,160.486110
2,3,3.525681e+07,96.312424,520.500055,72.499997


# 其他药物 

In [18]:
df_ot = pd.read_csv("/public/hanl/jupyter_dir/database/mimicIV20/clean/medication/other_drug/other_drug.csv")

def custom_map(value):
    return map_dict.get(value, value)

map_dict = {'Insulin - Regular':"Insulin",'Insulin - Glargine':"Insulin",'Insulin - Humalog':"Insulin",'Insulin - NPH':"Insulin",
            'Insulin - 70/30':"Insulin",'Insulin - Novolog':"Insulin",'Insulin - Humalog 75/25':"Insulin",'Insulin - U500':"Insulin",
           'Nicardipine 40mg':'Nicardipine'}
df_ot["label"] = df_ot["label"].map(custom_map)

df_ot['label'] = df_ot['label'].str.replace(r'\(.*\)', '')
df_ot['label'] = df_ot['label'].str.rstrip(' 0123456789/-.%')


<ipython-input-18-7b4d959f544a>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ot = pd.read_csv("/public/hanl/jupyter_dir/database/mimicIV20/clean/medication/other_drug/other_drug.csv")
<ipython-input-18-7b4d959f544a>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_ot['label'] = df_ot['label'].str.replace(r'\(.*\)', '')


In [19]:
df_ot["starttime"] = pd.to_datetime(df_ot["starttime"])
df_ot = pd.merge(df_ot,df_time,how="right",on="stay_id")
# df_ot = df_ot[(df_ot["starttime"]<=df_ot["saki_onset_a1d"])&(df_ot["starttime"]>=df_ot["saki_onset"])]
df_ot = df_ot[[ 'stay_id', 'starttime', "label", 'amount',"groupHPD" ]]
df_ot.columns = [ 'stay_id', 'starttime', "drug", 'amount',"groupHPD" ]
df_ot.head(1)

,stay_id,starttime,drug,amount,groupHPD
0,30003598,2189-04-18 12:07:00,Furosemide,20.000001,1


In [20]:
df_sum = df_ot.groupby(["stay_id","drug"]).agg({"amount":"sum"}).reset_index()
df_sum.head(1)

,stay_id,drug,amount
0,30003598,Calcium Gluconate,2.0


In [21]:
# 药物使用率 - 总体
df_sum["is_med"] = 1
df_stat_01 = df_sum.pivot(index="stay_id",columns="drug",values="is_med").reset_index()
df_stat_01 = pd.merge(df_stat_01,df_type_filt,how="right",on="stay_id")
df_stat_01 = df_stat_01.fillna(0)
# 统计总体的用药率
count_ones = df_stat_01.eq(1).sum()
df_stat_01_all = pd.DataFrame(count_ones / len(df_stat_01) * 100).T
df_stat_01_all

,stay_id,ACD-A Citrate,Acetaminophen-IV,Acetylcysteine,Adenosine,Alteplase,Aminocaproic acid,Amiodarone,Angiotensin II,Argatroban,...,Sodium Acetate,Sodium Bicarbonate,Thiamine,Tirofiban,Treprostinil,Vasopressin,Vecuronium,Verapamil,Vitamin K,groupHPD
0,0.0,2.33397,38.574157,0.360704,0.891152,0.297051,0.148525,12.900488,0.10609,0.742627,...,0.021218,10.014853,10.311903,0.127307,0.021218,13.855294,2.33397,0.10609,5.049862,27.540844


In [22]:
# 药物使用率 - 亚型
df_sum["is_med"] = 1
df_stat_01 = df_sum.pivot(index="stay_id",columns="drug",values="is_med").reset_index()
df_stat_01 = pd.merge(df_stat_01,df_type_filt,how="right",on="stay_id")
df_stat_01 = df_stat_01.fillna(0)
df_stat_01 = df_stat_01.groupby('groupHPD').mean()*100
df_stat_01.head()

,stay_id,ACD-A Citrate,Acetaminophen-IV,Acetylcysteine,Adenosine,Alteplase,Aminocaproic acid,Amiodarone,Angiotensin II,Argatroban,...,Rocuronium,Sodium Acetate,Sodium Bicarbonate,Thiamine,Tirofiban,Treprostinil,Vasopressin,Vecuronium,Verapamil,Vitamin K
groupHPD,,,,,,,,,,,,,,,,,,,,,
1,3.488627e+09,3.235747,38.135593,0.385208,0.924499,0.385208,0.308166,17.103236,0.308166,0.847458,...,3.929122,0.000000,12.557781,9.630200,0.077042,0.077042,19.799692,2.234206,0.000000,7.627119
2,3.508386e+09,0.261866,38.952537,0.261866,0.589198,0.261866,0.098200,10.245499,0.032733,0.556465,...,3.044190,0.032733,5.597381,9.656301,0.130933,0.000000,7.790507,2.193126,0.130933,2.880524
3,3.521878e+09,16.666667,36.944444,1.111111,3.333333,0.277778,0.000000,20.277778,0.000000,1.944444,...,6.388889,0.000000,38.333333,18.333333,0.277778,0.000000,43.888889,3.888889,0.277778,14.166667


In [23]:
# 合并
df_stat_01_add = pd.concat([df_stat_01_all,df_stat_01],axis=0)
df_stat_01_add["groupHPD"] = [0,1,2,3]
# 去除总SA-AKI患者小于10%的药物
first_row = df_stat_01_add.iloc[0]
columns_to_drop = first_row[first_row <= 10].index
df_stat_01_add = df_stat_01_add.drop(columns=columns_to_drop)
df_stat_01_add["groupHPD"] = [0,1,2,3]
df_stat_01_add = df_stat_01_add[df_stat_01_add.columns[df_stat_01_add.iloc[0].argsort()[::-1]]] # dataframe按第一行从大到小排序列
df_stat_01_add 

,Magnesium Sulfate,Potassium Chloride,Propofol,Furosemide,Insulin,Fentanyl,Heparin Sodium,KCL,Calcium Gluconate,Phenylephrine,...,K Phos,Nitroglycerin,Vasopressin,Amiodarone,Ranitidine,Na Phos,Haloperidol,Thiamine,Sodium Bicarbonate,groupHPD
0,73.329090,71.865054,69.700827,65.202631,65.054106,59.028220,56.842775,55.781880,55.272650,43.284532,...,17.377467,16.847019,13.855294,12.900488,12.518566,11.945682,11.563760,10.311903,10.014853,0
1,68.798151,68.875193,65.100154,71.648690,66.332820,60.708783,57.781202,55.315871,55.238829,43.682589,...,15.793529,16.409861,19.799692,17.103236,12.018490,12.249615,10.400616,9.630200,12.557781,1
2,76.039280,74.369885,72.373159,63.175123,64.517185,56.693944,56.432079,57.414075,53.813421,41.505728,...,18.559738,17.512275,7.790507,10.245499,13.355155,11.685761,12.635025,9.656301,5.597381,2
3,66.666667,61.388889,63.611111,59.166667,65.000000,72.777778,56.944444,43.611111,67.777778,56.944444,...,13.055556,12.777778,43.888889,20.277778,7.222222,13.055556,6.666667,18.333333,38.333333,3


# 合并

In [45]:
df_merge = pd.concat([df_anti,df_fluid,df_di,df_ot],axis=0) #df_vaso,
df_merge['drug'] = df_merge['drug'].str.replace(r'\(.*\)', '')
df_merge['drug'] = df_merge['drug'].str.rstrip(' 0123456789/-.%')

df_sum = df_merge.groupby(["stay_id","drug"]).agg({"amount":"sum"}).reset_index()
df_sum.head(1)

<ipython-input-45-f905969aa3f8>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_merge['drug'] = df_merge['drug'].str.replace(r'\(.*\)', '')


,stay_id,drug,amount
0,30003598.0,Calcium Gluconate,2.0


In [46]:
# 药物使用率 - 总体
df_sum["is_med"] = 1
df_stat_01 = df_sum.pivot(index="stay_id",columns="drug",values="is_med").reset_index()
df_stat_01 = pd.merge(df_stat_01,df_type_filt,how="right",on="stay_id")
df_stat_01 = df_stat_01.fillna(0)
# 统计总体的用药率
count_ones = df_stat_01.eq(1).sum()
df_stat_01_all = pd.DataFrame(count_ones / len(df_stat_01) * 100).T
df_stat_01_all

,stay_id,*NF* Moxifloxacin,ACD-A Citrate,AMOXicillin Oral Susp,Acetaminophen-IV,Acetylcysteine,Adenosine,Albumin,Alteplase,Amikacin,...,Verapamil,Vital,Vital High Protein,Vitamin K,Vivonex,ceFAZolin,ciprofloxacin,moxifloxacin,vancomycin,groupHPD
0,0.0,0.021218,2.33397,0.042436,38.574157,0.360704,0.891152,36.049226,0.297051,0.360704,...,0.10609,1.273074,3.119032,5.049862,0.636537,0.021218,0.021218,0.021218,0.212179,27.540844


In [47]:
# 药物使用率 - 亚型
df_sum["is_med"] = 1
df_stat_01 = df_sum.pivot(index="stay_id",columns="drug",values="is_med").reset_index()
df_stat_01 = pd.merge(df_stat_01,df_type_filt,how="right",on="stay_id")
df_stat_01 = df_stat_01.fillna(0)
df_stat_01 = df_stat_01.groupby('groupHPD').mean()*100
df_stat_01.head()

,stay_id,*NF* Moxifloxacin,ACD-A Citrate,AMOXicillin Oral Susp,Acetaminophen-IV,Acetylcysteine,Adenosine,Albumin,Alteplase,Amikacin,...,Vecuronium,Verapamil,Vital,Vital High Protein,Vitamin K,Vivonex,ceFAZolin,ciprofloxacin,moxifloxacin,vancomycin
groupHPD,,,,,,,,,,,,,,,,,,,,,
1,3.488627e+09,0.077042,3.235747,0.000000,38.135593,0.385208,0.924499,39.368259,0.385208,0.616333,...,2.234206,0.000000,1.771957,3.312789,7.627119,1.155624,0.077042,0.000000,0.000000,0.000000
2,3.508386e+09,0.000000,0.261866,0.065466,38.952537,0.261866,0.589198,33.191489,0.261866,0.196399,...,2.193126,0.130933,1.145663,2.945990,2.880524,0.392799,0.000000,0.032733,0.032733,0.327332
3,3.521878e+09,0.000000,16.666667,0.000000,36.944444,1.111111,3.333333,48.333333,0.277778,0.833333,...,3.888889,0.277778,0.555556,3.888889,14.166667,0.833333,0.000000,0.000000,0.000000,0.000000


In [48]:
# 合并
df_stat_01_add = pd.concat([df_stat_01_all,df_stat_01],axis=0)
df_stat_01_add["groupHPD"] = [0,1,2,3]
# 去除总SA-AKI患者小于10%的药物
first_row = df_stat_01_add.iloc[0]
columns_to_drop = first_row[first_row <= 10].index
df_stat_01_add = df_stat_01_add.drop(columns=columns_to_drop)
df_stat_01_add["groupHPD"] = [0,1,2,3]
df_stat_01_add = df_stat_01_add[df_stat_01_add.columns[df_stat_01_add.iloc[0].argsort()[::-1]]] # dataframe按第一行从大到小排序列
df_stat_01_add.to_csv("mimic_drug_useage.csv",index=False)
df_stat_01_add.T

,0,1,2,3
NaCl,97.178018,97.996918,96.628478,98.888889
Solution,77.530236,74.961479,78.232406,80.833333
Dextrose,77.487800,83.513097,73.617021,88.611111
Magnesium Sulfate,73.329090,68.798151,76.039280,66.666667
Potassium Chloride,71.865054,68.875193,74.369885,61.388889
Propofol,69.700827,65.100154,72.373159,63.611111
Vancomycin,69.276469,75.192604,65.106383,83.333333
Furosemide,65.202631,71.648690,63.175123,59.166667
Insulin,65.054106,66.332820,64.517185,65.000000
Fentanyl,59.028220,60.708783,56.693944,72.777778


In [49]:
# 取C2,C3亚型差异top10的药物
row_difference = (df_stat_01_add.iloc[3] - df_stat_01_add.iloc[2]).abs()
top_10_columns = row_difference.nlargest(60).index
df_stat_01_add[top_10_columns].T

,0,1,2,3
Norepinephrine,36.940378,46.918336,29.230769,66.388889
Vasopressin,13.855294,19.799692,7.790507,43.888889
Sodium Bicarbonate,10.014853,12.557781,5.597381,38.333333
Midazolam,33.078718,37.981510,28.772504,51.944444
Fresh Frozen Plasma,14.300870,20.338983,9.689034,31.666667
Packed Red Blood Cells,38.489285,48.228043,32.536825,53.888889
Pantoprazole,32.654360,38.674884,28.150573,49.166667
Piperacillin-Tazobactam,24.485466,30.046225,20.425532,38.888889
Vancomycin,69.276469,75.192604,65.106383,83.333333
CefePIME,29.896032,35.516179,25.990180,42.777778
